## Sprint 3 Reflection

### Creating the Group Design
[Link to Figma Template](https://www.figma.com/design/0fkn4vCGZvJeiaE47suJdW/Flocker-Room?node-id=0-1&node-type=canvas&t=dfH6VcYav35drSg6-0)

This is where I created the entire design along with another member, which would serve as the template for which we would use to create our code. It laid the foundations, and was a great experience for teaching me how to format and aesthetically design a website.

### Backend Development (Went over this with Mr. Mortensen)
<img src="../../../images/backend.png">

The backend was obviously the most valuable lesson in Sprint 3, I learned how to collect and store data, and this is something that essentially allows a website to actually function. 

If you click on channels, you will be able to see the full structure/directory of the database (.db), which I can navigate through to see all the different storages of information for the flocker website.

It stores posts, and additional information.

<img src="../../../images/posts.png">

This is an example of posting, the posts option is on the flocker_frontend (frontend of the website), and then once you post it, it gets stored in the databse (backend). If the backend didn't exist, all posts would be displayed, but would go away once the request to the website was refreshed.

### My Favorite (and largest) Contribution
<img src="../../../images/aichatroom.png">

This is an example of my favorite contribution. I used a Gemini API as the AI for the user to communicate with. To test out the AI, I used to Postman to communicate with the API endpoint, sending "POST" requests to test it out. I eventually got it working, and was even able to make dynamically generated messages, that sounded human. Overall, this was my faborite contribution!